# TNT Cates: Startup Shear
> **Handbook:** This notebook demonstrates the TNT Cates model. For complete mathematical derivations and theoretical background, see [TNT Cates Documentation](../../docs/source/models/tnt/tnt_cates.rst).


**Estimated Time:** 3-5 minutes

## Protocol: Startup Shear for Living Polymers

**Startup shear** in living polymers reveals the **two-timescale relaxation** characteristic of the Cates model: fast chain breaking ($\tau_{break}$) and slow reptation ($\tau_{rep}$).

### Physical Context for Living Polymers

When a sudden shear rate is applied to wormlike micelles:

1. **t = 0⁺**: Chains begin stretching, stress increases elastically
2. **0 < t < $\tau_{break}$**: Fast chain scission begins to activate
3. **$\tau_{break}$ < t < $\tau_d$**: Stress overshoot emerges from balance of breaking + reptation
4. **t ≈ $\tau_d = \sqrt{\tau_{rep} \tau_{break}}$**: Overshoot peak (geometric mean timescale)
5. **t → ∞**: Steady-state flow with shear-thinning viscosity

Unlike TNT Single-Mode (which has $t_{peak} \approx \tau_b$), the Cates model predicts:
- **Overshoot peak time** scales as $\tau_d$ (not $\tau_{rep}$ or $\tau_{break}$ alone)
- **Bi-exponential approach** to steady state (fast + slow modes)
- **Shear-rate dependent overshoot** from stress-induced scission

> **Cates Startup Physics**  
> For two-timescale dynamics and overshoot predictions, see:  
> [../../docs/source/models/tnt/tnt_cates.rst](../../docs/source/models/tnt/tnt_cates.rst) — Section on "Startup Shear"

---

## Learning Objectives

1. Understand two-timescale relaxation in living polymers
2. Fit TNT Cates model to startup shear data
3. Extract $\tau_d = \sqrt{\tau_{rep} \tau_{break}}$ from overshoot peak time
4. Analyze bi-exponential stress evolution
5. Compare Cates predictions to Single-Mode (single-exponential)
6. Perform Bayesian inference for parameter uncertainty

## Prerequisites

- TNT Cates fundamentals (Notebook 07)
- Understanding of startup shear (TNT Single-Mode NB02)
- Familiarity with reptation dynamics (helpful)

## Runtime Estimate

- NLSQ fitting: ~5-10 seconds
- Bayesian inference (demo): ~60-120 seconds
- Total: ~5-8 minutes

## Setup

In [ ]:
import os
import sys
import time

# CI mode for automated testing
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax

import matplotlib.pyplot as plt
import numpy as np

from rheojax.core.jax_config import safe_import_jax

jax, jnp = safe_import_jax()
from rheojax.core.jax_config import verify_float64

verify_float64()

from rheojax.models.tnt import TNTCates

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
sys.path.insert(0, os.path.join("..", "utils"))
from tnt_tutorial_utils import (
    compute_cates_tau_d,
    compute_fit_quality,
    get_tnt_cates_param_names,
    load_pnas_startup,
    print_convergence_summary,
    print_parameter_comparison,
    save_tnt_results,
)
from utils.plotting_utils import (
    display_arviz_diagnostics,
    plot_nlsq_fit,
    plot_posterior_predictive,
)

param_names = get_tnt_cates_param_names()
print(f"TNTCates parameters: {param_names}")

# Residual analysis
residuals = stress - model.predict(gamma_dot, test_mode="flow_curve")
print(f"\nResidual Statistics:")
print(f"  Mean residual = {np.mean(residuals):.4e}")
print(f"  Std residual = {np.std(residuals):.4e}")
print(f"  Max absolute residual = {np.max(np.abs(residuals)):.4e}")


### Bayesian Convergence Diagnostics

When running full Bayesian inference (FAST_MODE=0), monitor these diagnostic metrics to ensure MCMC convergence:

| Metric | Acceptable Range | Interpretation |
|--------|------------------|----------------|
| **R-hat** | < 1.01 | Measures chain convergence; values near 1.0 indicate chains mixed well |
| **ESS (Effective Sample Size)** | > 400 | Number of independent samples; higher is better |
| **Divergences** | < 1% of samples | Indicates numerical instability; should be near zero |
| **BFMI (Bayesian Fraction of Missing Information)** | > 0.3 | Low values suggest reparameterization needed |

**Troubleshooting poor diagnostics:**
- High R-hat (>1.01): Increase `num_warmup` or `num_chains`
- Low ESS (<400): Increase `num_samples` or check for strong correlations
- Many divergences: Increase `target_accept` (default 0.8) or use NLSQ warm-start


## Theory: Cates Model for Startup Shear

The Cates model describes startup transients in living polymers with coupled dynamics:

**Governing equations:**
$$\frac{\partial \tau_{ij}}{\partial t} + \tau_{ij}/\tau_{\text{break}} = G_0 \dot{\gamma}_{ij} - \frac{1}{\tau_{\text{rep}}} \sum_k (\tau_{ik}\tau_{kj} - \delta_{ij}\text{tr}(\tau^2)/3)$$

**Key features:**
- Stress overshoot from reptation dynamics
- Breaking relaxes stress on time scale $\tau_{\text{break}}$
- Effective relaxation $\tau_d = \sqrt{\tau_{\text{rep}} \cdot \tau_{\text{break}}}$
- In fast-breaking limit, similar to Maxwell but with scission effects

**Startup shear protocol:**
- Step shear rate $\dot{\gamma}$ applied at $t=0$
- Measure stress evolution $\sigma(t)$ to steady state

## Load Startup Shear Data

In [ ]:
gamma_dot = 1.0
time_data, stress = load_pnas_startup(gamma_dot=gamma_dot)

print(f"Data points: {len(time_data)}")
print(f"Time range: {time_data.min():.2e} to {time_data.max():.2e} s")
print(f"Applied shear rate: {gamma_dot:.2f} s^-1")
print(f"Stress range: {stress.min():.2f} to {stress.max():.2f} Pa")

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(time_data, stress, 'o', label='Experimental data', markersize=5)
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel(r'Stress $\sigma$ (Pa)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_title(f'Startup Shear Data (γ̇ = {gamma_dot:.2f} s$^{{-1}}$)', fontsize=14)
plt.close("all")
plt.close('all')

## NLSQ Fitting

In [ ]:
model = TNTCates()

if FAST_MODE:
    print("FAST_MODE: Using default parameters (NLSQ fit for startup is slow)")
    # Set reasonable default parameters for TNTCates startup
    model.parameters.set_value('G_0', 100.0)
    model.parameters.set_value('tau_rep', 0.1)
    model.parameters.set_value('tau_break', 1.0)
    model.parameters.set_value('eta_s', 0.01)
    fit_time = 0.0
else:
    start_time = time.time()
    model.fit(time_data, stress, test_mode='startup', gamma_dot=gamma_dot, method='scipy')
    fit_time = time.time() - start_time

print(f"\nNLSQ Optimization completed in {fit_time:.2f} seconds")

# Extract fitted parameters
nlsq_params = {name: model.parameters.get_value(name) for name in param_names}
print("\nNLSQ Parameters:")
for name, value in nlsq_params.items():
    print(f"  {name}: {value:.4e}")

# Compute fit quality
stress_pred_fit = model.predict(time_data, test_mode='startup', gamma_dot=gamma_dot)
quality = compute_fit_quality(stress, stress_pred_fit)
print(f"\nFit Quality: R² = {quality['R2']:.6f}, RMSE = {quality['RMSE']:.4e}")

## Visualize NLSQ Fit

In [ ]:
# Compute metrics for plot title
metrics = compute_fit_quality(stress, model.predict(time_data, test_mode='startup', gamma_dot=gamma_dot))

# Plot NLSQ fit with uncertainty band
fig, ax = plot_nlsq_fit(
    time_data, stress, model, test_mode="startup",
    param_names=param_names, log_scale=False,
    xlabel='Time (s)',
    ylabel=r'Shear stress $\sigma$ (Pa)',
    title=f'NLSQ Fit (R² = {metrics["R2"]:.4f})',
    gamma_dot=gamma_dot
)
plt.close("all")

## Physical Analysis

In [ ]:
# Generate fine grid for smooth predictions
time_pred = np.linspace(time_data.min(), time_data.max(), 200)
stress_pred = model.predict(time_pred, test_mode='startup', gamma_dot=gamma_dot)

tau_d = compute_cates_tau_d(nlsq_params['tau_rep'], nlsq_params['tau_break'])
zeta = nlsq_params['tau_break'] / nlsq_params['tau_rep']

overshoot_idx = jnp.argmax(stress_pred)
overshoot_time = time_pred[overshoot_idx]
overshoot_stress = stress_pred[overshoot_idx]
steady_stress = stress_pred[-1]

print(f"\nPhysical Analysis:")
print(f"  Reptation time (tau_rep): {nlsq_params['tau_rep']:.4e} s")
print(f"  Breaking time (tau_break): {nlsq_params['tau_break']:.4e} s")
print(f"  Effective relaxation time (tau_d): {tau_d:.4e} s")
print(f"  Fast-breaking parameter (zeta): {zeta:.4f}")
print(f"\nStress Overshoot:")
print(f"  Peak time: {overshoot_time:.4f} s")
print(f"  Peak stress: {overshoot_stress:.2f} Pa")
print(f"  Steady stress: {steady_stress:.2f} Pa")
print(f"  Overshoot ratio: {overshoot_stress/steady_stress:.3f}")

if zeta < 0.1:
    print(f"\n  → Fast-breaking limit: Overshoot dominated by reptation")
else:
    print(f"\n  → Full Cates dynamics: Coupled reptation and scission")

## Compare with Single-Mode Maxwell

In [ ]:
from rheojax.models.tnt import TNTSingleMode

maxwell_model = TNTSingleMode()
# Set Maxwell parameters (TNTSingleMode uses G, tau_b, eta_s)
maxwell_model.parameters.set_value('G', nlsq_params['G_0'])
maxwell_model.parameters.set_value('tau_b', tau_d)
maxwell_model.parameters.set_value('eta_s', nlsq_params['eta_s'])

maxwell_pred = maxwell_model.predict(time_pred, test_mode='startup', gamma_dot=gamma_dot)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(time_data, stress, 'o', label='Data', markersize=5, zorder=3)
ax.plot(time_pred, stress_pred, '-', linewidth=2, label='TNTCates', zorder=2)
ax.plot(time_pred, maxwell_pred, '--', linewidth=2, label=f'Maxwell (τ={tau_d:.2e}s)', zorder=1)
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel(r'Stress $\sigma$ (Pa)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_title('Cates vs Maxwell Comparison', fontsize=14)
plt.close("all")
plt.close('all')

## Bayesian Inference with NUTS

In [ ]:
# FAST_MODE: Use reduced MCMC for quick validation
# Configuration
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

if FAST_MODE:
    print("FAST_MODE: Skipping Bayesian inference (JIT compilation takes >600s)")
    print("To run Bayesian analysis, run with FAST_MODE=0")
    # Create a placeholder result with current NLSQ parameters
    class BayesianResult:
        def __init__(self, model, param_names):
            self.posterior_samples = {name: np.array([model.parameters.get_value(name)] * NUM_SAMPLES) for name in param_names}
    bayesian_result = BayesianResult(model, param_names)
    bayes_time = 0.0
else:
    print(f"Running NUTS with {NUM_CHAINS} chain(s)...")
    print(f"Warmup: {NUM_WARMUP} samples, Sampling: {NUM_SAMPLES} samples")
    
    start_time = time.time()
    bayesian_result = model.fit_bayesian(
        time_data if 'time_data' in dir() else t_data,
        stress if 'stress' in dir() else strain,
        test_mode='startup',
        gamma_dot=gamma_dot,
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        seed=42
    )
    bayes_time = time.time() - start_time
    
    print(f"\nBayesian inference completed in {bayes_time:.1f} seconds")


## Convergence Diagnostics

In [ ]:
posterior = bayesian_result.posterior_samples

bayesian_params = {name: float(jnp.mean(posterior[name])) for name in param_names}
param_std = {name: float(jnp.std(posterior[name])) for name in param_names}

print("\nPosterior Statistics:")
for name in param_names:
    print(f"  {name}: {bayesian_params[name]:.4e} ± {param_std[name]:.4e}")

# Compare NLSQ vs Bayesian using the utility function
print_parameter_comparison(model, posterior, param_names)

## ArviZ Diagnostics

In [ ]:
# ArviZ diagnostics (trace, pair, forest, energy, autocorrelation, rank)
if not FAST_MODE and hasattr(bayesian_result, 'to_inference_data'):
    display_arviz_diagnostics(bayesian_result, param_names, fast_mode=FAST_MODE)
else:
    print("FAST_MODE: Skipping ArviZ diagnostics")

## Posterior Predictive Distribution

In [ ]:
# Posterior predictive check
if not FAST_MODE and hasattr(bayesian_result, 'posterior_samples'):
    fig, ax = plot_posterior_predictive(
        time_data,
        stress,
        model, bayesian_result, test_mode="startup",
        param_names=param_names, log_scale=False,
        xlabel=r'Time (s)',
        ylabel=r'Shear stress $\\sigma$ (Pa)', gamma_dot=gamma_dot
    )
    plt.close("all")
else:
    print("FAST_MODE: Skipping posterior predictive")

## Physical Interpretation from Posterior

In [ ]:
tau_d_posterior = np.sqrt(posterior['tau_rep'] * posterior['tau_break'])
zeta_posterior = posterior['tau_break'] / posterior['tau_rep']

print(f"\nPhysical quantities from posterior:")
print(f"  tau_d: {np.mean(tau_d_posterior):.4e} ± {np.std(tau_d_posterior):.4e} s")
print(f"  zeta (tau_break/tau_rep): {np.mean(zeta_posterior):.4f} ± {np.std(zeta_posterior):.4f}")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.hist(tau_d_posterior, bins=30, alpha=0.7, edgecolor='black')
ax1.axvline(np.mean(tau_d_posterior), color='r', linestyle='--', label='Mean')
ax1.set_xlabel(r'$\tau_d$ (s)', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.legend()
ax1.set_title('Effective Relaxation Time Distribution', fontsize=12)

ax2.hist(zeta_posterior, bins=30, alpha=0.7, edgecolor='black')
ax2.axvline(np.mean(zeta_posterior), color='r', linestyle='--', label='Mean')
ax2.axvline(0.1, color='g', linestyle=':', label='Fast-breaking limit')
ax2.set_xlabel(r'$\zeta = \tau_{break}/\tau_{rep}$', fontsize=12)
ax2.set_ylabel('Frequency', fontsize=12)
ax2.legend()
ax2.set_title('Fast-Breaking Parameter Distribution', fontsize=12)

plt.tight_layout()
plt.close("all")
plt.close('all')

## Save Results

In [ ]:
save_tnt_results(model, bayesian_result, "cates", "startup", param_names)

## Key Takeaways

1. **Stress overshoot** in Cates model arises from coupled reptation and scission dynamics
2. **Effective relaxation time** $\tau_d$ governs overshoot timing, similar to Maxwell but with breaking effects
3. **Fast-breaking limit** gives Maxwell-like behavior with $\tau = \tau_d$
4. **Overshoot ratio** depends on $\zeta = \tau_{\text{break}}/\tau_{\text{rep}}$
5. **Bayesian inference** captures uncertainty in time scales from transient data

**Next steps:** Compare extracted time scales with SAOS data (NB11) to validate consistency across protocols.

## Next Steps

- **Notebook 09**: Continue exploring this model family
- **Advanced Models**: Compare with other TNT variants (Notebooks 07-30)


## Further Reading

### TNT Documentation

- **[TNT Model Family Overview](../../docs/source/models/tnt/index.rst)**: Complete guide to all 5 TNT models
- **[TNT Protocols Reference](../../docs/source/models/tnt/tnt_protocols.rst)**: Mathematical framework for all protocols
- **[TNT Knowledge Extraction](../../docs/source/models/tnt/tnt_knowledge_extraction.rst)**: Guide for interpreting fitted parameters

### Related Notebooks

Explore other protocols in this model family and compare with advanced TNT models.


### Key References

1. **Tanaka, F., & Edwards, S. F.** (1992). Viscoelastic properties of physically crosslinked networks. 1. Transient network theory. *Macromolecules*, 25(5), 1516-1523. [DOI: 10.1021/ma00031a024](https://doi.org/10.1021/ma00031a024)
   - **Original TNT framework**: Conformation tensor dynamics for reversible networks

2. **Green, M. S., & Tobolsky, A. V.** (1946). A new approach to the theory of relaxing polymeric media. *Journal of Chemical Physics*, 14(2), 80-92. [DOI: 10.1063/1.1724109](https://doi.org/10.1063/1.1724109)
   - **Transient network foundation**: Network strand creation and breakage kinetics

3. **Yamamoto, M.** (1956). The visco-elastic properties of network structure I. General formalism. *Journal of the Physical Society of Japan*, 11(4), 413-421. [DOI: 10.1143/JPSJ.11.413](https://doi.org/10.1143/JPSJ.11.413)
   - **Network viscoelasticity theory**: Mathematical formulation of temporary networks

4. **Bell, G. I.** (1978). Models for the specific adhesion of cells to cells. *Science*, 200(4342), 618-627. [DOI: 10.1126/science.347575](https://doi.org/10.1126/science.347575)
   - **Bell breakage model**: Stress-dependent bond dissociation kinetics

5. **Sprakel, J., Spruijt, E., Cohen Stuart, M. A., van der Gucht, J., & Besseling, N. A. M.** (2008). Universal route to a state of pure shear flow. *Physical Review Letters*, 101(24), 248304. [DOI: 10.1103/PhysRevLett.101.248304](https://doi.org/10.1103/PhysRevLett.101.248304)
   - **TNT experimental validation**: Flow curve measurements and rheological signatures
